# Clean and combine data

## Setup

#### Import dependencies

In [1]:
import pandas as pd
import numpy as np
import csv

#### Import csvs

In [2]:
fed_funds = pd.read_csv('../original_resources/fed_funds.csv')
treasury = pd.read_csv('../original_resources/10YT_minus_2YT.csv')

## Fed funds by quarter

#### Print head

In [3]:
fed_funds.head()

,date,value
0,1954-07-01,1.13
1,1954-07-02,1.25
2,1954-07-03,1.25
3,1954-07-04,1.25
4,1954-07-05,0.88


#### Rename value column

In [4]:
fed_funds.rename(columns={' value': 'rate'}, inplace=True)

#### Add column to track the quarter

In [5]:
fed_funds['quarter'] = ''

#### Convert date column from string to datetime object

In [6]:
fed_funds.date = pd.to_datetime(fed_funds.date, format='%Y-%m-%d')

#### Populate column based on date value

In [7]:
for i in range(len(fed_funds)):
    
    # Define month and year variable
    month = fed_funds.date[i].month
    year = str(fed_funds.date[i].year)
    
    # Define conditional to choose the quarter
    if month in [1, 2, 3]:
        quarter = 'Q1'
    
    elif month in [4, 5, 6]:
        quarter = 'Q2'
    
    elif month in [7, 8, 9]:
        quarter = 'Q3'
    
    else:
        quarter = 'Q4'
        
    # Put it altogether
    fed_funds.iloc[i, fed_funds.columns.get_loc('quarter')] = year + quarter

#### Group by quarter and calculate average value

In [8]:
fed_funds_avg = fed_funds.groupby('quarter').mean().reset_index()

#### Add column to fed_funds_avg showing percent change from previous quarter

In [9]:
fed_funds_avg['fed_funds_percent_change_prev_quarter'] = ''

#### Rename rate column

In [10]:
fed_funds_avg.rename(columns={'rate': 'fed_funds_avg_rate'}, inplace=True)

#### Populate percent change from previous quarter column

In [13]:
for i in range(len(fed_funds_avg)):
    
    if i != 0:
        current_value = fed_funds_avg.iloc[i, fed_funds_avg.columns.get_loc('fed_funds_avg_rate')]
        previous_value = fed_funds_avg.iloc[i-1, fed_funds_avg.columns.get_loc('fed_funds_avg_rate')]
        
        fed_funds_avg.iloc[i, fed_funds_avg.columns.get_loc('fed_funds_percent_change_prev_quarter')] = (current_value - previous_value)/previous_value

In [14]:
fed_funds_avg.head()

,quarter,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter
0,1954Q3,1.028478,
1,1954Q4,0.988370,-0.0389981
2,1955Q1,1.345000,0.360827
3,1955Q2,1.501099,0.116059
4,1955Q3,1.936522,0.290069


#### Drop first row

In [15]:
fed_funds_avg.drop(index=0, inplace=True)

#### Group by quarter and calculate standard deviation

In [16]:
fed_funds_std = fed_funds.groupby('quarter').std().reset_index()

#### Rename column to reflect it shows standard deviation

In [17]:
fed_funds_std.rename(columns={'rate': 'fed_funds_st_dev_rate'}, inplace=True)

#### Merge two dataframes

In [18]:
fed_funds_final = pd.merge(fed_funds_avg, fed_funds_std, how='inner', on='quarter')

#### Export to csv

In [19]:
fed_funds_final.to_csv('../resources/fed_funds_final.csv', index=False)

## Treasury by quarter

#### Print head

In [20]:
treasury.head()

,DATE,T10Y2YM
0,1976-06-01,0.80
1,1976-07-01,0.98
2,1976-08-01,1.14
3,1976-09-01,1.17
4,1976-10-01,1.43


#### Rename columns

In [21]:
treasury.rename(columns={'DATE': 'date', 'T10Y2YM': '10YT_minus_2YT'}, inplace=True)

#### Add column to track the quarter

In [22]:
treasury['quarter'] = ''

#### Convert date column from string to datetime object

In [23]:
treasury.date = pd.to_datetime(treasury.date, format='%Y-%m-%d')

#### Populate quarter column based on date value

In [24]:
for i in range(len(treasury)):
    
    # Define month and year variable
    month = treasury.date[i].month
    year = str(treasury.date[i].year)
    
    # Define conditional to choose the quarter
    if month in [1, 2, 3]:
        quarter = 'Q1'
    
    elif month in [4, 5, 6]:
        quarter = 'Q2'
    
    elif month in [7, 8, 9]:
        quarter = 'Q3'
    
    else:
        quarter = 'Q4'
        
    # Put it altogether
    treasury.iloc[i, treasury.columns.get_loc('quarter')] = year + quarter

#### Group by quarter and calculate average value

In [25]:
treasury_avg = treasury.groupby('quarter').mean().reset_index()

#### Rename column to reflect value is an average

In [26]:
treasury_avg.rename(columns={'10YT_minus_2YT': '10YT_minus_2YT_avg'}, inplace=True)

#### Add column to treasury_avg to show percent change from previous quarter

In [27]:
treasury_avg['10YT_minus_2YT_percent_change_prev_quarter'] = ''

#### Populate percent change column

In [138]:
for i in range(len(treasury_avg)):
    
    if i != 0:
        current_value = treasury_avg.iloc[i, treasury_avg.columns.get_loc('10YT_minus_2YT_avg')]
        previous_value = treasury_avg.iloc[i-1, treasury_avg.columns.get_loc('10YT_minus_2YT_avg')]
        
        treasury_avg.iloc[i, treasury_avg.columns.get_loc('percent_change_prev_quarter_10YT_minus_2YT')] = (current_value - previous_value)/previous_value

#### Drop first row

In [140]:
treasury_avg.drop(index=0, inplace=True)

#### Export to csv

In [ ]:
treasury_avg.to_csv('../resources/10YT_minus_2YT_final.csv', index=False)